In [41]:
import pandas as pd
import numpy as np
import processing.b_feats.util
import processing.processing_utils
import importlib
from constants import LEVELS, START, MAX_DELAY
from rlenv.time_triggers import *
from compress_pickle import dump, load
import sys
import os
os.chdir('A:/ebay/')
sys.path.append('repo/processing/b_feats/test/')
import meta_time_feats_test  
importlib.reload(meta_time_feats_test)
importlib.reload(processing.b_feats.time_funcs)
importlib.reload(processing.b_feats.util)
setup_complex_lstg = meta_time_feats_test.setup_complex_lstg
add_event = meta_time_feats_test.add_event
NEW_LSTG = 'new_lstg'
ACCEPTANCE = 'accept'
EXPIRE_LSTG = 'expire_lstg'
CENSORED = 'censored'

In [32]:
events = setup_complex_lstg(None, meta=1, leaf=1)

# add a group where there are 2 lstgs and no threads
offer = {
    'lstg': 23,
    'clock': 300
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=3, leaf=1)
offer['clock'] = 325
events = add_event(events, offer, trigger_type=EXPIRE_LSTG, meta=3, leaf=1)

offer = {
    'lstg': 24,
    'clock': 350
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=3, leaf=1)
offer['clock'] = 355
events = add_event(events, offer, trigger_type=EXPIRE_LSTG, meta=3, leaf=1)

# add a group where there are 2 lstgs, several threads, and 1 bin
offer = {
    'lstg': 25,
    'clock': 360
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=4, leaf=1)
offer['clock'] = 365
offer['byr'] = True
offer['price'] = 35
offer['thread'] = 1
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['price'] = 45
offer['clock'] = 370
offer['thread'] = 2
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['price'] = 55
offer['clock'] = 375
offer['thread'] = 3
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['price'] = 60
offer['clock'] = 380
offer['thread'] = 4
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['clock'] = 385
offer['lstg'] = 26
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=4, leaf=1)

offer['thread'] = 1
offer['price'] = 50
offer['clock'] = 387
offer['byr'] = True
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['thread'] = 2
offer['price'] = 100
offer['clock'] = 389
events = add_event(events, offer, trigger_type=ACCEPTANCE, meta=4, leaf=1)
offer['thread'] = 1
offer['byr'] = False
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)

offer['clock'] = 400
offer['byr'] = False
offer['lstg'] = 25
offer['price'] = 60
offer['thread'] = 4
events = add_event(events, offer, trigger_type=ACCEPTANCE, meta=4, leaf=1)
offer['thread'] = 3
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)
offer['thread'] = 2
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)
offer['thread'] = 1
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)
events.index = events.index.droplevel(['cndtn', 'leaf'])
events.byr = events.byr.astype(bool)
events.reject = events.reject.astype(bool)
events.accept = events.accept.astype(bool)
events.censored = events.censored.astype(bool)
events.flag = events.flag.astype(bool)
events.price = events.price.astype(np.int64)
events.clock = events.clock.astype(np.int64)

In [42]:
clock_df = events.clock.unstack()
delay = pd.DataFrame(index=clock_df.index)
delay[0] = 0
for i in range(1, 8):
    if i in clock_df.columns:
        delay[i] = clock_df[i] - clock_df[i-1]
        if i in [2, 4, 6]:
            max_delay = MAX_DELAY['slr']
            censored = delay[i] > MAX_DELAY['slr']
        elif i != 7:
            censored = delay[i] > MAX_DELAY['byr']
            max_delay = MAX_DELAY['byr']
        else:
            censored = delay.index
            max_delay = 1
        delay.loc[censored, i] = np.NaN
        delay[i] = delay[i] / max_delay
events['delay'] = delay.rename_axis('index', axis=1).stack()

In [43]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(events['delay'])

meta  lstg  thread  index
1     1     0       0        0.000000
            1       1             NaN
                    2        0.000058
                    3        0.000008
                    4        0.000174
                    5        0.000004
            2       1             NaN
                    2        0.000000
      2     0       0        0.000000
      1     3       1             NaN
            2       3        0.000025
      3     0       0        0.000000
      1     2       4        0.000058
      2     1       1             NaN
      3     1       1             NaN
      2     2       1             NaN
      1     2       5        0.000012
            0       1        0.000103
      3     1       2        0.000116
      2     1       2        0.000116
      4     0       0        0.000000
      2     1       3        0.000004
            2       2        0.000145
                    3        0.000004
            1       4        0.000058
      3     1       3   

In [29]:
feats = processing.b_feats.util.get_cat_feats(events, levels=['meta'], feat_ind=3)
feats.isna().any().any()

lstg group
           bin  thread
meta lstg             
1    1     0.0     3.0
     2     0.0     2.0
     3     0.0     1.0
     4     0.0     1.0
     5     0.0     2.0
     6     0.0     2.0
     7     0.0     1.0
     8     0.0     1.0
     9     0.0     1.0
     10    0.0     0.0
     11    0.0     0.0
3    23    0.0     0.0
     24    0.0     0.0
4    25    0.0     4.0
     26    1.0     2.0
level group
      bin  thread
meta             
1     0.0    14.0
3     0.0     0.0
4     1.0     6.0


False

In [30]:
feats

,meta_bin,meta_first_offer_25,meta_first_offer_50,meta_first_offer_75,meta_first_offer_100
lstg,,,,,
1,0.0,0.30,0.50,0.60,0.90
2,0.0,0.30,0.50,0.60,0.90
3,0.0,0.30,0.50,0.60,0.90
4,0.0,0.30,0.50,0.60,0.90
5,0.0,0.30,0.50,0.50,0.90
6,0.0,0.50,0.50,0.60,0.90
7,0.0,0.30,0.50,0.55,0.90
8,0.0,0.30,0.50,0.55,0.75
9,0.0,0.50,0.50,0.60,0.90


In [196]:
actual

,meta_lstgs_open,meta_lstg_inds,meta_threads,meta_slr_offers,meta_byr_offers,meta_accepts,meta_accept_norm_25,meta_accept_norm_50,meta_accept_norm_75,meta_accept_norm_100
lstg,,,,,,,,,,
1,1,10,14,9,13,8,0.5,0.7,0.85,1.00
2,2,10,16,11,15,7,0.5,0.7,0.85,1.00
3,3,10,17,12,16,7,0.5,0.8,0.85,1.00
4,3,10,17,13,17,7,0.6,0.8,0.85,1.00
5,1,10,16,13,16,7,0.5,0.8,0.85,1.00
6,2,10,16,12,16,7,0.6,0.8,0.85,1.00
7,3,10,16,13,17,7,0.5,0.7,0.80,0.95
8,1,10,17,11,17,7,0.5,0.7,0.80,1.00
9,2,10,17,12,17,7,0.5,0.7,0.80,1.00


In [72]:
total_lstgs_index = pd.MultiIndex.from_tuples([(1, 2), (1, 1), (2, 3), (2, 1)], names=['meta','leaf'])
total_lstgs = pd.DataFrame(index=total_lstgs_index, data={'tot': [4, 3, 1, 9]})
lstg_index = pd.MultiIndex.from_tuples([(1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 1, 6), (1, 1, 7), (1, 1, 8), (1, 1, 9), (2, 3, 1), (2, 1, 3), (2, 1, 2)], names=['meta', 'leaf', 'lstg'])
lstg_index

MultiIndex([(1, 2, 1),
            (1, 2, 2),
            (1, 2, 3),
            (1, 1, 6),
            (1, 1, 7),
            (1, 1, 8),
            (1, 1, 9),
            (2, 3, 1),
            (2, 1, 3),
            (2, 1, 2)],
           names=['meta', 'leaf', 'lstg'])

In [73]:
total_lstgs.reindex(lstg_index)

tot
meta leaf lstg     
1    2    1       4
          2       4
          3       4
     1    6       3
          7       3
          8       3
          9       3
2    3    1       1
     1    3       9
          2       9

In [74]:
total_lstgs_index = pd.Index([1, 2])
total_lstgs = pd.DataFrame(index=total_lstgs_index, data={'tot': [4, 3]})
lstg_index = pd.MultiIndex.from_tuples([(1, 1), (1, 2), (1, 3), (2, 3), (2, 2)], names=['meta', 'lstg'])
lstg_index

MultiIndex([(1, 1),
            (1, 2),
            (1, 3),
            (2, 3),
            (2, 2)],
           names=['meta', 'lstg'])

In [75]:
total_lstgs.reindex(lstg_index, level='meta')

tot
meta lstg     
1    1       4
     2       4
     3       4
2    3       3
     2       3

In [77]:
t = pd.Index([1, 2, 3, 4])
p = pd.Index([2, 1, 3, 4])

In [78]:
t.equals(p)

False

In [107]:
t = pd.DataFrame(data={'clock': [4, 9, 11, 1, 2, 15, 15], 'censored': [2, 2, 2, 2, 2, 2, 1]})

In [109]:
t.sort_values(['clock', 'censored'])

,clock,censored
3,1,2
4,2,2
0,4,2
1,9,2
2,11,2
6,15,1
5,15,2


In [20]:
path = 'A:/ebay/data/chunks/m1.gz'
d = load(path)
L, T, O = [d[k] for k in ['listings', 'threads', 'offers']]

In [21]:
L = processing.processing_utils.categories_to_string(L)

KeyError: 'product'

In [23]:
L.columns

Index(['meta', 'leaf', 'cndtn', 'start_date', 'end_time', 'start_price',
       'arrival_rate', 'flag'],
      dtype='object')

In [31]:
L['arrival_rate'].max()
L['']

86400.00001186563

In [28]:
O.columns

Index(['clock', 'price', 'accept', 'reject', 'censored'], dtype='object')

In [29]:
T.columns

Index(['start_time', 'byr_hist', 'bin'], dtype='object')

In [30]:
T['byr_hist'].head()

lstg      thread
57184026  1           0
49123002  1          19
          2           1
93457629  1          51
47950695  1         194
Name: byr_hist, dtype: int64

In [25]:
levels = ['meta', 'leaf', 'cndtn']
events = processing.b_feats.util.create_events(L, T, O, levels)

In [26]:
events.columns

Index(['accept', 'censored', 'clock', 'price', 'reject', 'byr', 'flag',
       'start_price'],
      dtype='object')

In [84]:
events = setup_complex_lstg(None, meta=1, leaf=1)
events = setup_complex_lstg(events, meta=2, leaf=1, starter=11)
events.index = events.index.droplevel(['cndtn', 'leaf'])
events = events.xs(1, level='index')

accept    byr  censored  clock   flag  price  reject  \
meta lstg thread index                                                         
1    1    0      0       False  False     False     15  False      0   False   
          1      1       False   True     False     20  False     50   False   
                 2       False  False     False     30  False     80   False   
                 3       False   True     False     40  False     60   False   
                 4       False  False     False     70  False     75   False   
...                        ...    ...       ...    ...    ...    ...     ...   
2    19   1      3       False   True     False    285  False     95   False   
                 4        True   True     False    285  False     95   False   
     22   0      0       False  False     False    290  False      0   False   
     21   0      1       False  False     False    295  False      0   False   
     22   0      1       False  False     False    295  False      0   False   

                       start_price  
meta lstg thread index              
1    1    0      0             100  
          1      1             100  
                 2             100  
                 3             100  
                 4             100  
...                            ...  
2    19   1      3             100  
                 4             100  
     22   0      0             100  
     21   0      1             100  
     22   0      1             100  

[114 rows x 8 columns]

In [60]:
t = pd.DataFrame({'a': [2, 4, 5, 6], 'b':[False, True, False, True]})

In [61]:
d = t.copy()

In [62]:
d.loc[d['b'], 'a'] = 20

In [63]:
print(d)

    a      b
0   2  False
1  20   True
2   5  False
3  20   True


In [64]:
t

,a,b
0,2,False
1,4,True
2,5,False
3,6,True
